In [5]:
# import Math library
import math
from IPython import display

# Importing matplotlib for better graphical visualization
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import seaborn as sns

# import numpy for numerical computing
import numpy as np
# import pandas for dataframe
import pandas as pd

import missingno as msno
from sklearn import metrics
from sklearn.preprocessing import scale, StandardScaler, normalize
from sklearn import preprocessing
from sklearn import decomposition

# import PCA 
from sklearn.decomposition import PCA
# import kmeans for clustering 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import pickle
from collections import Counter

# import python plot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.1f}'.format

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
# Libraries of model to make predictions and to validate the models working
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
from sklearn.model_selection import  cross_val_score
from sklearn.externals import joblib 

In [6]:
# loading online retail dataset dataset
data = pd.read_excel("/Users/ruchitha/Desktop/DataMining_Projects/Online-retail-Analysis/clean_data.xlsx")

In [7]:
# calculating total cost of the product purchased
total_cost = data['Quantity'] * data['UnitPrice']
data_new = data[(data['total_cost'] >0) & (data['total_cost'] < 100)]
spend_label = []
for i in range(0,len(data_new),1):
    if data_new['total_cost'].iloc[i] < 11:
        spend_label.append(0)
    elif data_new['total_cost'].iloc[i] >30:
        spend_label.append(2)
    else:
        spend_label.append(1)
# adding an additonal spent_label feature
data_new['spend_label'] = spend_label
#  Creating dummies for description
dummy_1 = pd.get_dummies(data=data_new['Description'])
new_ds_1 = pd.concat([data_new, dummy_1], axis=1)
# drop the columns that have been now been replaced  by derived features 
unwanted_cols = ['Quantity','StockCode','InvoiceDate','InvoiceNo','UnitPrice','Description','spend_label','CustomerID']
new_ds_1 = new_ds_1.drop(unwanted_cols, axis=1)
new_ds_1 = new_ds_1.reset_index(drop=True)
# selecting the features for model training
new_ds_3 = new_ds_1.copy()
# deleting the target colum from features set
del new_ds_3['Country']
# taget for training model
ydata2 = new_ds_1['Country']
# convert ydata2 to integer
ydata2 = ydata2.astype('category')
normalized_data2 = preprocessing.normalize(new_ds_3)

**GradientBoostin: LightGBM**

In [15]:
# convert string label to float & make new sets for LightGBM
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import train_test_split
lb = LabelEncoder()
ydata_float = lb.fit_transform(ydata2)
# splittng model data
xtrain, xtest, ytrain, ytest = train_test_split(normalized_data2,ydata_float,test_size = 0.3, random_state = 8418) 
train_data=lgb.Dataset(xtrain,label=ytrain)
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':37,
    'metric': 'multi_logloss',
    'learning_rate': 0.05,
    'max_depth': 7,
    'num_leaves': 17,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.6,
    'bagging_freq': 17}

lgb_cv = lgb.cv(params, train_data, num_boost_round=10000, nfold=5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=100)

nround = lgb_cv['multi_logloss-mean'].index(np.min(lgb_cv['multi_logloss-mean']))
lgbm_model2 = lgb.train(params, train_data, num_boost_round=nround)
#predicting on test set
ypred=lgbm_model2.predict(xtest)
predictions = []

for x in ypred:
    predictions.append(np.argmax(x))


[20]	cv_agg's multi_logloss: 1.13298 + 0.0012676
[40]	cv_agg's multi_logloss: 0.777325 + 0.00124495
[60]	cv_agg's multi_logloss: 0.657756 + 0.00140435
[80]	cv_agg's multi_logloss: 0.612428 + 0.00166689
[100]	cv_agg's multi_logloss: 0.593498 + 0.00189162
[120]	cv_agg's multi_logloss: 0.583033 + 0.00198964
[140]	cv_agg's multi_logloss: 0.577493 + 0.00200542
[160]	cv_agg's multi_logloss: 0.574449 + 0.00205914
[180]	cv_agg's multi_logloss: 0.572362 + 0.00214975
[200]	cv_agg's multi_logloss: 0.570935 + 0.00220593
[220]	cv_agg's multi_logloss: 0.570193 + 0.00222563
[240]	cv_agg's multi_logloss: 0.569572 + 0.00230161
[260]	cv_agg's multi_logloss: 0.571507 + 0.00441704
[280]	cv_agg's multi_logloss: 0.571669 + 0.00501266
[300]	cv_agg's multi_logloss: 0.569287 + 0.00229977
[320]	cv_agg's multi_logloss: 0.569241 + 0.00229556
[340]	cv_agg's multi_logloss: 0.569307 + 0.00226408
[360]	cv_agg's multi_logloss: 0.570741 + 0.00424561
[380]	cv_agg's multi_logloss: 0.569914 + 0.00275463
[400]	cv_agg's mul

In [16]:
accuracy = accuracy_score(ytest, predictions)
# model perfomance scores
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print('Recall:', recall_score(ytest, predictions,average="micro")*100,"%")
print('Precision:', precision_score(ytest, predictions,average="micro")*100,"%")

Accuracy: 89.20%
Recall: 89.1980283598477 %
Precision: 89.1980283598477 %
